# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,elverum,60.8819,11.5623,3.84,86,59,2.21,NO,1712871702
1,1,ushuaia,-54.8000,-68.3000,7.81,61,0,11.32,AR,1712871614
2,2,port-aux-francais,-49.3500,70.2167,5.90,89,100,3.34,TF,1712871702
3,3,uige,-7.6087,15.0613,21.09,99,100,1.27,AO,1712871702
4,4,west island,-12.1568,96.8225,25.99,89,40,2.57,CC,1712871702


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    title = "Humidity Map",
    frame_width=700,
    frame_height=500,
    size= "Humidity",
    scale=0.75,
    color="City"
)

# Display the map
humidity_plot

:Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_df = city_data_df[(city_data_df['Max Temp'] >=21) & (city_data_df['Max Temp'] <= 27)]
ideal_weather_df = ideal_temp_df[(ideal_temp_df['Wind Speed'] > 4.5) & (ideal_temp_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,i-n-salah,27.1935,2.4607,23.01,21,0,7.86,DZ,1712871702
11,11,praia,14.9215,-23.5087,22.30,73,0,5.66,CV,1712871702
16,16,pisco,-13.7000,-76.2167,26.03,65,0,5.66,PE,1712871703
97,97,crane,31.3974,-102.3501,26.53,14,0,5.34,US,1712871709
229,229,lompoc,34.6391,-120.4579,22.69,72,0,5.66,US,1712871718
305,305,tamanrasset,22.7850,5.5228,25.95,8,0,5.14,DZ,1712871726
354,354,callao,-12.0667,-77.1500,26.16,65,0,5.14,PE,1712871592
435,435,bubaque,11.2833,-15.8333,24.22,71,0,9.19,GW,1712871736
549,549,jirja,26.3383,31.8916,24.12,31,0,5.14,EG,1712871746


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.drop(columns=['City_ID', 'Max Temp','Cloudiness', 'Wind Speed', 'Date'])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
0,elverum,60.8819,11.5623,86,NO,
1,ushuaia,-54.8000,-68.3000,61,AR,
2,port-aux-francais,-49.3500,70.2167,89,TF,
3,uige,-7.6087,15.0613,99,AO,
4,west island,-12.1568,96.8225,89,CC,
...,...,...,...,...,...,...
550,tongliao,43.6125,122.2653,70,CN,
551,tobol'sk,58.2000,68.2667,75,RU,
552,sur,22.5667,59.5289,44,OM,
553,micheweni,-4.9667,39.8333,84,TZ,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius=10000
limit=20
categories = "accommodation.hotel"
params = {
    "limit": limit,
    "categories": categories,
    "apiKey": geoapify_key   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
elverum - nearest hotel: Scandic Central Elverum
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
port-aux-francais - nearest hotel: Keravel
uige - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
edinburgh of the seven seas - nearest hotel: No hotel found
mount gambier - nearest hotel: The Old Mount Gambier Gaol
gaalkacyo - nearest hotel: Citycenter وسط المدينة
i-n-salah - nearest hotel: باجودة
puerto natales - nearest hotel: Dorotea Patagonia Hostel
tazovsky - nearest hotel: Тазовчанка
praia - nearest hotel: Casa Sodadi
lata - nearest hotel: Residencial Botânico
murun-kuren - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
kuvandyk - nearest hotel: Отдыхающая локомотивных бригад
pisco - nearest hotel: La Portada
alaghsas - nearest hotel: Hôtel Zagado
nova vicosa - nearest hotel: Pousada Pontal da Barra
happy valley-goose bay - nearest hotel: Hotel North Two
nouadhibou - nearest hotel: Hotel valenci

,City,Lat,Lng,Humidity,Country,Hotel Name
0,elverum,60.8819,11.5623,86,NO,Scandic Central Elverum
1,ushuaia,-54.8000,-68.3000,61,AR,Apart Hotel Aires del Beagle
2,port-aux-francais,-49.3500,70.2167,89,TF,Keravel
3,uige,-7.6087,15.0613,99,AO,No hotel found
4,west island,-12.1568,96.8225,89,CC,Cocos Village Bungalows
...,...,...,...,...,...,...
550,tongliao,43.6125,122.2653,70,CN,No hotel found
551,tobol'sk,58.2000,68.2667,75,RU,Ямская
552,sur,22.5667,59.5289,44,OM,Sur Hotel
553,micheweni,-4.9667,39.8333,84,TZ,No hotel found


In [7]:
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
0,elverum,60.8819,11.5623,86,NO,Scandic Central Elverum
1,ushuaia,-54.8000,-68.3000,61,AR,Apart Hotel Aires del Beagle
2,port-aux-francais,-49.3500,70.2167,89,TF,Keravel
3,uige,-7.6087,15.0613,99,AO,No hotel found
4,west island,-12.1568,96.8225,89,CC,Cocos Village Bungalows
...,...,...,...,...,...,...
550,tongliao,43.6125,122.2653,70,CN,No hotel found
551,tobol'sk,58.2000,68.2667,75,RU,Ямская
552,sur,22.5667,59.5289,44,OM,Sur Hotel
553,micheweni,-4.9667,39.8333,84,TZ,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    title = "Nearest Hotels",
    frame_width=700,
    frame_height=500,
    size= "Humidity",
    scale=0.5,
    color="City",
    hover_cols="Hotel Name"
)

# Display the map
hotel_plot

:Points   [Lng,Lat]   (City,Humidity,Hotel Name)